# Figure 6. d) analysis

This notebook generates the data used to create the Fig. 6 d) circle packing plot. The actual plot was created using D3.js. An Observable notebook that implements the plotting can be found [here](https://observablehq.com/@duncster94/fig-6-d).

In [1]:
import json
import numpy as np
import pandas as pd
from pathlib import Path
from scipy.cluster.hierarchy import linkage, fcluster, maxdists

Import BIONIC features, Costanzo bioprocesses, IntAct protein complexes, and TS genes.

In [2]:
# import BIONIC features
features = pd.read_csv("../data/methods/yeast_BIONIC_features.csv", index_col=0)

# import Costanzo bioprocesses
with Path("../data/standards/Costanzo-bioprocess.json").open("r") as f:
    bioprocesses = json.load(f)

# import IntAct complex standard
with Path("../data/standards/yeast-IntAct-complex-modules.json").open("r") as f:
    complexes = json.load(f)

# import list of genes for which we have TS alleles for
ts_genes = list(pd.read_csv("../data/chemical-genetics/TS-genes.txt", header=None, sep="\n")[0].values)

Subset `features` to only contain genes in `ts_genes` and genes in the "Glycosylation, protein folding/targeting, cell wall biosynthesis" bioprocess.

In [3]:
# get genes in bioprocess
bioprocess = "Glycosylation, protein folding/targeting, cell wall biosynthesis"
bioprocess_genes = bioprocesses[bioprocess]

# find common genes
ts_genes_in_biop = np.intersect1d(ts_genes, bioprocess_genes)
common_genes = np.intersect1d(ts_genes_in_biop, list(features.index))

# subset `features`
features = features.reindex(common_genes)

Cluster the BIONIC features, but don't extract clusters yet.

In [4]:
link = linkage(features.values, method="average", metric="cosine")

Next we will extract clusters from `link` at two different levels. The first (lowest) level will contain clusters that correspond to protein complexes (i.e. adaptive cluter thresholds), where as the second (highest) level will simply be a flat threshold.

First we define a function to identify protein complexes in `link` (the first, lowest level). For each protein complex present in the "Glycosylation, protein folding/targeting, cell wall biosynthesis" bioprocess, this function identifies the clustering threhold that best matches said complex. For genes which do not fit into a known complex, a flat clustering threshold given by `highest_thresh` will be used instead.

In [5]:
def get_first_level_clusters(complexes):

    # find maximum allowable clustering threshold
    max_thresh = np.max(maxdists(link))
    
    assignments = {}
    highest_thresh = 0  # highest threshold found so far

    # iterate over protein complexes and identify best matching cluters
    for complex, comp_genes in complexes.items():

        # skip if the protein complex has fewer than 2 members in `common_genes`
        if len(np.intersect1d(comp_genes, common_genes)) < 2:
            continue

        # keep track of the best cluster-complex overlap score (Jaccard)
        best_jaccard = 0

        # iterate over cluster thresholds `t` and find thresholds that best match
        # the known complex given by `comp_genes`
        for thresh in np.linspace(0, max_thresh, num=1000):

            # extract clusters
            clusters = fcluster(link, thresh, criterion="distance")

            # get unique indices corresponding to `clusters`
            labels = np.unique(clusters)

            # iterate over each cluster assignment and compare cluster with complex
            for label in labels:

                # get gene indices corresponding to cluster given by `label`
                cluster = np.argwhere(clusters == label).flatten()

                # ignore clusters with fewer than 2 members
                if len(cluster) < 2:
                    continue

                # get genes from indices
                cluster_genes = common_genes[cluster]

                # compute Jaccard score numerator and denominator by comparing cluster with complex
                numer = len(np.intersect1d(cluster_genes, comp_genes))
                denom = len(np.union1d(cluster_genes, comp_genes))

                if denom == 0:
                    jaccard = 0
                else:
                    jaccard = numer / denom

                # check the Jaccard score is the best so far, and that the complex 
                # is considered "captured" (Jaccard score >= 0.5)
                if jaccard > best_jaccard and jaccard >= 0.5:
                    if thresh > highest_thresh:
                        highest_thresh = thresh
                    best_jaccard = jaccard

                    # update the cluster assignment for the given complex
                    assignments[complex] = cluster

    # create an array of indices corresponding to the captured complexes
    gene_assignments = np.zeros(len(common_genes))
    print("Captured complexes:")
    for i, (complex, idxs) in enumerate(assignments.items()):
        print(complex)
        gene_assignments[idxs] = i + 1

    # combine `gene_assignments` with clusters obtained by extracting at `highest_thresh`
    # we add 0.1 to `highest_thresh` so that there are fewer split clusters (better visualization)
    final_clusters = (
        np.max(gene_assignments) + 1 + fcluster(link, highest_thresh + 0.1, criterion="distance")
    )
    final_clusters[np.nonzero(gene_assignments)] = gene_assignments[np.nonzero(gene_assignments)]

    return final_clusters

Generate clusters from the first and second levels. We also include a "zeroth" level cluster set that corresponds to all singleton clusters. This is done simply to align the indices from clsuter assignments in the first and second levels to the genes in `common_genes`.

In [6]:
first_level_clusters = get_first_level_clusters(complexes)

# cluster threshold 0.9 is arbitrary and chosen simply to show higher order organization
second_level_clusters = fcluster(link, 0.9, criterion="distance")

# get singleton cluster assignments
zeroth_level_clusters = fcluster(link, 0, criterion="distance")

# create the final set of clusters
clusters = [zeroth_level_clusters, first_level_clusters, second_level_clusters]

Captured complexes:
CPX-1643
CPX-3056
CPX-1269


Finally, we create a dictionary in the correct format for D3.js to use for visualization. This is accomplished by recursively building the dictionary from the top down. We also map genes from ORFs to gene names.

In [7]:
# import name mapping dictionary
with Path("../data/chemical-genetics/yeast-orf2name-mapper.json").open("r") as f:
    orf2name_mapper = json.load(f)
mapped_genes = np.array([orf2name_mapper[gene] if gene in orf2name_mapper else gene for gene in common_genes])

# define recursive dictionary builder
def create_data_structure_recursive(clusters, subset, idx):

    if idx + 1 > len(clusters):
        names = mapped_genes[subset]
        return [{"name": name, "value": 1} for name in names]

    cluster = clusters[idx]
    cluster_ = cluster[subset]
    labels = np.unique(cluster_)
    children = []

    for label in labels:

        # get indices of `cluster` which equal label
        new_subset = np.argwhere(clusters[idx] == label).flatten()
        child = create_data_structure_recursive(clusters, new_subset, idx + 1)
        if isinstance(child, list):
            children += child
        else:
            children.append(child)
    return {"name": "", "children": children}

# define recursive wrapper function
def create_data_structure(clusters):

    # reverse `clusters`
    clusters = clusters[::-1]

    return create_data_structure_recursive(clusters, np.arange(len(clusters[0])), 0)

Create the dictionary used in the D3.js visualization. This matches the "clusters_Glycosylation, protein folding-targeting, cell wall biosynthesis.json" file used in the D3.js visualization. Implementation of the visualization can be found [here](https://observablehq.com/@duncster94/fig-6-d).

In [9]:
data = create_data_structure(clusters)
data

{'name': '',
 'children': [{'name': '',
   'children': [{'name': '',
     'children': [{'name': 'CDS1', 'value': 1}, {'name': 'COP1', 'value': 1}]},
    {'name': '',
     'children': [{'name': 'GDI1', 'value': 1}, {'name': 'MRS6', 'value': 1}]},
    {'name': '', 'children': [{'name': 'FRQ1', 'value': 1}]}]},
  {'name': '',
   'children': [{'name': '',
     'children': [{'name': 'ALG14', 'value': 1},
      {'name': 'ALG13', 'value': 1}]},
    {'name': '',
     'children': [{'name': 'SEC63', 'value': 1},
      {'name': 'SEC62', 'value': 1}]},
    {'name': '',
     'children': [{'name': 'GPI18', 'value': 1},
      {'name': 'PGA1', 'value': 1}]},
    {'name': '',
     'children': [{'name': 'NUS1', 'value': 1},
      {'name': 'SEC59', 'value': 1},
      {'name': 'DPM1', 'value': 1}]},
    {'name': '', 'children': [{'name': 'PMI40', 'value': 1}]},
    {'name': '', 'children': [{'name': 'ALG2', 'value': 1}]},
    {'name': '',
     'children': [{'name': 'ERD2', 'value': 1},
      {'name': 'ERO